# Exercice 1 : Charger un modèle quantifié et préparer le fine-tuning

## Objectif 
Charger un modèle de langage pré-entraîné, appliquer une quantification en 4 bits et préparer le modèle pour le fine-tuning avec LoRA.

## Étapes 
1. Choisir un modèle pré-entraîné (par exemple : `gpt2`, `LLaMA`, ou `falcon`).
2. Quantifier le modèle en 4 bits avec la fonction appropriée.
3. Préparer le modèle pour le fine-tuning avec LoRA en ciblant les couches `q_proj` et `v_proj`.

## Question 
- Quels sont les avantages de la quantification en 4 bits par rapport à la quantification classique en 8 bits ?


In [21]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

In [22]:
model_name = "gpt2"
model = AutoModelForCausalLM.from_pretrained(model_name)

# Quantification du modele en 4 bits
model_quantifize = prepare_model_for_kbit_training(model, kbit = 4)

tokenizer = AutoTokenizer.from_pretrained(model_name)

print(model)

ImportError: 
AutoModelForCausalLM requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.
